# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [24]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [25]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [26]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [27]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [28]:
from cassandra.cluster import Cluster

try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)


#### Create Keyspace

In [29]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS songs 
        WITH REPLICATION = 
            { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [30]:
try:
    session.set_keyspace('songs')
except Exception as e:
    print(e)

## Query 1
### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
For Query one we will make table music_app_history, the (session_id, item_in_session) were used as a partition key because the queries will filter by these column. 

In [32]:


query = "CREATE TABLE IF NOT EXISTS music_app_history "
query = query + "(session_id int,item_in_session int,artist text, song_title text, length float,  PRIMARY KEY ((session_id, item_in_session)))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [36]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_app_history (session_id, item_in_session ,artist , song_title , length  )"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        line[5] = float(line[5])
        line[3] = int(line[3] )
        line[-3] = int(line[-3] )
      
        session.execute(query, (line[-3],line[3],line[0],line[-2],line[5]))


#### Do a SELECT to verify that the data have been inserted into each table

In [37]:
query = "select artist , song_title , length from music_app_history WHERE session_id=338 and  item_in_session=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## Query 2
### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

For Query two we will make table music_app_usage, the columns (user_id,session_id) were used as a partition key because the queries will filter by these column.  item_in_session was used as clustering columns to help make up a unique key.

In [38]:

query = "CREATE TABLE IF NOT EXISTS music_app_usage "
query = query + "(user_id int,session_id int,item_in_session int, song_title text,artist text,name  text,  PRIMARY KEY ((user_id,session_id),item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [40]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_app_usage (user_id ,session_id ,item_in_session , song_title ,artist ,name )"
        query = query + " VALUES (%s, %s, %s, %s,%s, %s)"


        session.execute(query, (int(line[-1]),int(line[-3]),int(line[3]),line[-2],line[0],line[1]+" " +line[4]))

                    

In [41]:
query = "select artist , song_title ,name from music_app_usage WHERE session_id=182 and  user_id=10"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Query
### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
For Query three we will make table query = "CREATE TABLE IF NOT EXISTS users "
, the columns song_title were used as a partition key because the queries will filter by this column. name was used as clustering columns to help make up a unique key.

In [42]:


query = "CREATE TABLE IF NOT EXISTS users "
query = query + "( song_title text, name text, session_id int, item_in_session int, PRIMARY KEY (song_title,name))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [43]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users ( song_title ,name ,session_id , item_in_session )"
        query = query + " VALUES (%s, %s, %s, %s)"

        session.execute(query, (line[-2],line[1]+" "+line[4],int(line[-3]),int(line[3])))

                    

In [44]:

query = "select name from users WHERE  song_title='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.name)

Jacqueline Lynch
Sara Johnson
Tegan Levine


### Drop the tables before closing out the sessions

In [ ]:
query = "DROP TABLE music_app_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "DROP TABLE music_app_usage"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE users"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    

### Close the session and cluster connection¶

In [38]:
session.shutdown()
cluster.shutdown()